<a href="https://colab.research.google.com/github/bhuvanabasapur/CMPE-258/blob/master/Assignments/Assignment%205/Assignment_5_part_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

plt.style.use('seaborn-whitegrid')

import warnings
warnings.filterwarnings('ignore')

In [4]:
!pip install -q kaggle
from google.colab import files


In [5]:
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [6]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle competitions download -c new-york-city-taxi-fare-prediction 

  0% 0.00/960k [00:00<?, ?B/s]
100% 960k/960k [00:00<00:00, 65.7MB/s]
  0% 0.00/486 [00:00<?, ?B/s]
100% 486/486 [00:00<00:00, 409kB/s]
  0% 0.00/335k [00:00<?, ?B/s]
100% 335k/335k [00:00<00:00, 105MB/s]
100% 1.55G/1.56G [00:14<00:00, 112MB/s] 
100% 1.56G/1.56G [00:14<00:00, 114MB/s]


In [8]:
#unzip train.csv.zip
import zipfile
zf = '/content/train.csv.zip'
zip_ref = zipfile.ZipFile(zf, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [15]:
df = pd.read_csv('/content/train.csv', parse_dates=True, infer_datetime_format=True)
df = df.dropna()

In [16]:
df.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07,5.542348e+07
mean,1.134500e+01,-7.250973e+01,3.991982e+01,-7.251121e+01,3.992068e+01,1.685391e+00
std,2.071087e+01,1.284878e+01,9.642324e+00,1.278220e+01,9.633346e+00,1.327662e+00
min,-3.000000e+02,-3.442060e+03,-3.492264e+03,-3.442025e+03,-3.547887e+03,0.000000e+00
25%,6.000000e+00,-7.399207e+01,4.073493e+01,-7.399140e+01,4.073403e+01,1.000000e+00
50%,8.500000e+00,-7.398180e+01,4.075265e+01,-7.398015e+01,4.075316e+01,1.000000e+00
75%,1.250000e+01,-7.396708e+01,4.076713e+01,-7.396367e+01,4.076810e+01,2.000000e+00
max,9.396336e+04,3.457626e+03,3.408790e+03,3.457622e+03,3.537133e+03,2.080000e+02


# Feature Engineering

In [17]:
def prepare_time_features(df):
    df['pickup_datetime'] = df['pickup_datetime'].str.slice(0, 16)
    df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], utc=True, format='%Y-%m-%d %H:%M')
    df['hour_of_day'] = df.pickup_datetime.dt.hour
    df['month'] = df.pickup_datetime.dt.month
    df["year"] = df.pickup_datetime.dt.year
    df["weekday"] = df.pickup_datetime.dt.weekday
    return df

In [18]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 0.6213712 * 12742 * np.arcsin(np.sqrt(a))

In [19]:
def transform(data):
    # Distances to nearby airports, 
    jfk = (-73.7781, 40.6413)
    ewr = (-74.1745, 40.6895)
    lgr = (-73.8740, 40.7769)

    data['pickup_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                         data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_jfk'] = distance(jfk[1], jfk[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_ewr'] = distance(ewr[1], ewr[0], 
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_ewr'] = distance(ewr[1], ewr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    data['pickup_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                          data['pickup_latitude'], data['pickup_longitude'])
    data['dropoff_distance_to_lgr'] = distance(lgr[1], lgr[0],
                                           data['dropoff_latitude'], data['dropoff_longitude'])
    
    return data

In [20]:
df = df.sample(25000)

In [21]:
df = prepare_time_features(df)
df['distance_miles'] = distance(df.pickup_latitude, df.pickup_longitude,df.dropoff_latitude, df.dropoff_longitude)

In [22]:
df = transform(df)


In [23]:
df_train = df.drop(columns=['key','pickup_datetime'], axis =1)
df_train.head()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,hour_of_day,month,year,weekday,distance_miles,pickup_distance_to_jfk,dropoff_distance_to_jfk,pickup_distance_to_ewr,dropoff_distance_to_ewr,pickup_distance_to_lgr,dropoff_distance_to_lgr
31013456,8.5,-73.983132,40.755662,-73.954153,40.779232,1,20,9,2010,5,2.225242,13.333738,13.260599,11.014286,13.096709,5.896374,4.196746
52201418,7.5,-73.985397,40.748226,-73.992798,40.756687,1,0,1,2015,4,0.701280,13.134290,13.785628,10.701816,10.586685,6.157177,6.371567
10567337,12.5,-73.954811,40.773453,-73.985184,40.747513,1,22,3,2015,5,2.395595,13.001393,13.097407,12.882145,10.693646,4.234971,6.162685
34017589,6.5,-73.986239,40.762370,-73.969699,40.762924,1,16,2,2013,2,0.866429,13.741849,13.089648,11.068999,11.863110,5.958340,5.099897
48139024,8.5,-73.979112,40.763530,-73.993663,40.746932,5,9,9,2011,4,1.376654,13.497521,13.445846,11.438139,10.267790,5.577208,6.595793


In [25]:
df_train.to_csv('NYC_Dtata.csv')

In [2]:
df_train = pd.read_csv('NYC_Dtata.csv')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               25000 non-null  int64  
 1   fare_amount              25000 non-null  float64
 2   pickup_longitude         25000 non-null  float64
 3   pickup_latitude          25000 non-null  float64
 4   dropoff_longitude        25000 non-null  float64
 5   dropoff_latitude         25000 non-null  float64
 6   passenger_count          25000 non-null  int64  
 7   hour_of_day              25000 non-null  int64  
 8   month                    25000 non-null  int64  
 9   year                     25000 non-null  int64  
 10  weekday                  25000 non-null  int64  
 11  distance_miles           25000 non-null  float64
 12  pickup_distance_to_jfk   25000 non-null  float64
 13  dropoff_distance_to_jfk  25000 non-null  float64
 14  pickup_distance_to_ewr

In [3]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('fare_amount', axis=1),
                                                    df_train['fare_amount'], test_size=0.3, random_state = 42)


In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(17500, 17)
(17500,)
(7500, 17)
(7500,)


# XGBoost Model

In [5]:
import xgboost as xgb
params = {
    'max_depth': 8,
    'gamma' :0,
    'eta':.05, 
    'subsample': 1,
    'colsample_bytree': 0.9, 
    'objective':'reg:linear',
    'eval_metric':'rmse',
    'silent': 0
}


In [6]:
def XGBmodel(X_train,X_test,y_train,y_test,params):
    matrix_train = xgb.DMatrix(X_train,label=y_train)
    matrix_test = xgb.DMatrix(X_test,label=y_test)
    model=xgb.train(params=params,
                    dtrain=matrix_train,num_boost_round=5000, 
                    early_stopping_rounds=10,evals=[(matrix_train, 'train'),(matrix_test,'test')])
    return model
model = XGBmodel(X_train,X_test,y_train,y_test,params)

[10:32:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	train-rmse:13.8463	test-rmse:14.4225
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 10 rounds.
[1]	train-rmse:13.2213	test-rmse:13.7955
[2]	train-rmse:12.629	test-rmse:13.2074
[3]	train-rmse:12.0904	test-rmse:12.6834
[4]	train-rmse:11.5573	test-rmse:12.1501
[5]	train-rmse:11.0524	test-rmse:11.6456
[6]	train-rmse:10.5736	test-rmse:11.1736
[7]	train-rmse:10.1204	test-rmse:10.7343
[8]	train-rmse:9.69092	test-rmse:10.318
[9]	train-rmse:9.28517	test-rmse:9.92361
[10]	train-rmse:8.90151	test-rmse:9.55541
[11]	train-rmse:8.53867	test-rmse:9.20741
[12]	train-rmse:8.1966	test-rmse:8.88021
[13]	train-rmse:7.87373	test-rmse:8.57496
[14]	train-rmse:7.58069	test-rmse:8.30001
[15]	train-rmse:7.29219	test-rmse:8.0363
[16]	train-rmse:7.01919	test-rmse:7.78796
[17]	train-rmse:6.76203	test-r